# Dataloader:

In [ ]:
pip install --upgrade pydata-google-auth google-cloud-bigquery[pandas] -q

In [ ]:
pip install pandas-gbq -q

In [ ]:
import pydata_google_auth
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
gbq_credentials = pydata_google_auth.get_user_credentials(
                                        ["https://www.googleapis.com/auth/bigquery"],
                                        use_local_webserver=False)

project_id = "ki-datalake-experiments-7b57"

### Load dataset:

In [ ]:
# Input the insurance group class:
# E.g., "COMMERCIAL NA OPEN MARKET", "COMMERCIAL WW", "PERSONAL ACCIDENT", "PI US"
query = """
    SELECT p.PolicyRef, p.Syndicate, p.InceptionDate, p.ExpiryDate, p.GrossNetWrittenPremium_USD, p.Exposure_USD,
           c.LossDateFrom, c.Incurred_USD
    FROM `ki-datalake-prod-c82c.mymi_dump_live.Policy` p
    JOIN `ki-datalake-prod-c82c.mymi_dump_live.Claim` c
    ON p.PolicyRef = c.PolicyRef
    WHERE p.GroupClass = "COMMERCIAL NA OPEN MARKET"
    """

In [ ]:
ki_df = pd.read_gbq(query, project_id=project_id, dialect="standard", credentials=gbq_credentials)

### Clean dataset:

In [ ]:
ki_df.InceptionDate = pd.to_datetime(ki_df.InceptionDate) + pd.offsets.MonthBegin(0)
ki_df.ExpiryDate = pd.to_datetime(ki_df.ExpiryDate) + pd.offsets.MonthBegin(0)
ki_df.LossDateFrom = pd.to_datetime(ki_df.LossDateFrom) + pd.offsets.MonthBegin(0)

ki_df.GrossNetWrittenPremium_USD = np.round(ki_df.GrossNetWrittenPremium_USD)
ki_df.Exposure_USD = np.round(ki_df.Exposure_USD)
ki_df.Incurred_USD = np.round(ki_df.Incurred_USD)

ki_df = ki_df.loc[ki_df.Incurred_USD >= 0]
ki_df = ki_df.loc[ki_df.Exposure_USD > 0]
ki_df = ki_df.loc[ki_df.GrossNetWrittenPremium_USD > 1000]
ki_df = ki_df.loc[ki_df.Exposure_USD >= ki_df.Incurred_USD]
ki_df = ki_df.loc[ki_df.Exposure_USD >= ki_df.GrossNetWrittenPremium_USD]
ki_df = ki_df.loc[ki_df.LossDateFrom >= ki_df.InceptionDate]
ki_df = ki_df.loc[ki_df.ExpiryDate >= ki_df.LossDateFrom]

for policy in ki_df.PolicyRef.unique():
    risk = ki_df.loc[ki_df.PolicyRef == policy]
    if risk.GrossNetWrittenPremium_USD.nunique() > 1:
        for premium in risk.GrossNetWrittenPremium_USD.unique():
            ki_df.loc[(ki_df.GrossNetWrittenPremium_USD == premium) & (ki_df.PolicyRef == policy), "PolicyRef"] = policy + str(premium)
    
for policy in ki_df.PolicyRef.unique():
    risk = ki_df.loc[ki_df.PolicyRef == policy]
    if risk.Incurred_USD.sum() > risk.Exposure_USD.mean():
        ki_df = ki_df.loc[ki_df.PolicyRef != policy]

ki_df.sort_values(by='InceptionDate', inplace=True)
ki_df.head()

### Plot syndicate dataset:

In [ ]:
syndicate = "2987"
capital = 100_000_000

syndicate_df = ki_df.loc[ki_df.Syndicate == syndicate]

grouped_syndicate_df = syndicate_df.groupby(["PolicyRef", "InceptionDate", "ExpiryDate"],
                                            as_index=False).mean().drop(columns=["Incurred_USD"])
time = pd.date_range(start=syndicate_df.InceptionDate.min(), 
                     end=max(syndicate_df.InceptionDate.max(), syndicate_df.LossDateFrom.max()), freq="MS")
df = pd.DataFrame({"time": time})

cash = []
cash.append(capital)
for step in time:        
    claims = sum(syndicate_df.loc[syndicate_df.LossDateFrom == step].Incurred_USD)
    premiums = sum(grouped_syndicate_df.loc[grouped_syndicate_df.InceptionDate == step].GrossNetWrittenPremium_USD)
    try:
        cash.append(cash[-1] + premiums - claims)
    except:
        if premiums != 0:
            cash.append(premiums - claims)
        else:
            cash.append(None)

    if premiums != 0:
        capacity = sum(grouped_syndicate_df.loc[grouped_syndicate_df.InceptionDate == step].Exposure_USD)

df[str(syndicate)] = cash[1:]

df.plot(x="time", y=syndicate, kind="line")
plt.xlabel("Time")
plt.ylabel("Captial (USD)")
plt.title("Insurer Capital")
plt.show()

print("Syndicate:", syndicate)
print()
print("Total premiums:", grouped_syndicate_df.GrossNetWrittenPremium_USD.sum(), "USD")
print("Total claims:", syndicate_df.Incurred_USD.sum(), "USD")
print("Net capital:", df[syndicate].iloc[-1] - df[syndicate].iloc[0], "USD")
print()
print("Number of risks:", syndicate_df.PolicyRef.nunique())
print()
print("Earliest date:", syndicate_df.InceptionDate.min()) 
print("Latest date:", max(syndicate_df.InceptionDate.max(), syndicate_df.LossDateFrom.max()))
print()

### Determine dataset features:

In [ ]:
print("Whole market features:")
print()

monthly_market_capacity = grouped_syndicate_df.groupby(["InceptionDate"], as_index=False).sum().Exposure_USD
print("Minimum capacity in one month:", monthly_market_capacity.min(), "USD")
print("Maximum capacity in one month:", monthly_market_capacity.max(), "USD")
print()

monthly_n_risks = grouped_syndicate_df.InceptionDate.value_counts()
print("Minimum number of risks incepted in one month:", monthly_n_risks.min())
print("Maximum number of risks incepted in one month:", monthly_n_risks.max())
print()

print("Individual risk features:")
print()

risk_capacity = grouped_syndicate_df.Exposure_USD
print("Minimum capacity:", risk_capacity.min(), "USD")
print("Maximum capacity:", risk_capacity.max(), "USD")
print()

claims = syndicate_df.groupby(["PolicyRef"], as_index=False).sum().Incurred_USD
premiums = syndicate_df.groupby(["PolicyRef"], as_index=False).mean().GrossNetWrittenPremium_USD
reward = premiums - claims
net_reward = np.abs(claims - premiums)
print("Minimum net capital:", net_reward.min(), "USD")
print("Maximum net capital:", net_reward.max(), "USD")
print()

risk_length = syndicate_df.groupby(["PolicyRef", "InceptionDate", "ExpiryDate"], as_index=False).mean()
risk_length = risk_length.ExpiryDate.dt.to_period('M').view(int) - risk_length.InceptionDate.dt.to_period('M').view(int)
print("Minimum contract length:", risk_length.min(), "months")
print("Maximum contract length:", risk_length.max(), "months")

# Simulation:

In [ ]:
pip install abcEconomics -q

In [ ]:
import os
import shutil
import tensorflow as tf

from tensorflow import set_random_seed
from abcEconomics import Simulation
from risk import Risk
from insurer import Insurer
from dqn_insurer import DQNInsurer
from customer import Customer
from generators import insurer_generator, dqn_insurer_generator

### Reset working directory:

In [ ]:
for folder in ["model", "result", "data", "plot"]:
    if not os.path.exists("/home/jupyter/" + folder):
        os.makedirs("/home/jupyter/" + folder)
    else:
        shutil.rmtree("/home/jupyter/" + folder, ignore_errors=False, onerror=None)
        os.makedirs("/home/jupyter/" + folder)

### Set up customer agents from the dataset:

In [ ]:
risk_id = -1
customer_parameters = []
actual_premiums = {}
market_features = {}
for date in time:
    entry = date
    risk_ids = []
    market_capacity = 0
    for policy in syndicate_df.loc[syndicate_df.InceptionDate == entry].PolicyRef.unique():
        risk = syndicate_df.loc[ki_df.PolicyRef == policy]
        capacity = risk.Exposure_USD.unique().item()
        length = (risk.ExpiryDate.dt.to_period('M').view(int) - risk.InceptionDate.dt.to_period('M').view(int)).unique().item()    
        claims = risk.loc[risk.Incurred_USD != 0]
        claims = claims.groupby(["LossDateFrom"], as_index=False).sum()
        claims = claims.drop(columns=["GrossNetWrittenPremium_USD", "Exposure_USD"])
        claims = claims.to_records(index=False)
        
        risk = Risk(entry, risk_id, capacity, length, claims)
        customer_parameters.append({"risk": risk})

        actual_premium = grouped_syndicate_df.loc[grouped_syndicate_df.PolicyRef == policy].GrossNetWrittenPremium_USD.item()
        actual_premiums[str(risk)] = actual_premium
        
        risk_id += 1
        risk_ids.append(risk_id)
        market_capacity += capacity
        
    market_features.update({str(entry): {"risk_ids": risk_ids, "market_capacity": market_capacity}})

### Set up baseline insurer agents:

In [ ]:
# Input the number of insurers (n_insurers):
n_insurers = 1

In [ ]:
# Input a list of premium formulae for the insurers (premium_formulae):
def premium_formula(risk):
    """
    Calculates a premium (scalar) for insuring a risk.
    """
    actual_premium = actual_premiums[str(risk)]
    premium = int(np.abs(actual_premium * (1.0 + 0.0 * np.random.normal())))
    return premium       
        
premium_formulae = []
for n in range(n_insurers):
    premium_formulae.append(premium_formula)

In [ ]:
# Input a list of initial cash values for the insurers (insurer_cash_values):
def insurer_cash():
    """
    Determines an insurer's intial cash value (scalar).
    """
    cash = 100_000_000 * (1 + 0.0 * np.random.normal())
    return cash

insurer_cash_values = []
for n in range(n_insurers):
    insurer_cash_values.append(insurer_cash())

### Set up DQN insurer agents:

In [ ]:
# Input the number of DQN insurers (n_dqn_insurers):
n_dqn_insurers = 1

In [ ]:
# Input a list of algorithms for the DQN insurers (dqn_insurer_algorithms):
# E.g., ["DQN"], "DDQN"], ["DQNPER"], ["DDQNPER"] ["PASSIVE"], ["DQN", "PASSIVE"]
dqn_insurer_algorithms = ["DQNPER"]

In [ ]:
# Input a list of weights to initialise the DQN insurers (dqn_insurer_weights):
# E.g., [None], ["001_0"], [None, "025_0"]
dqn_insurer_weights = [None]

In [ ]:
# Input the dimensions of the state space for the DQN insurers (dqn_insurer_states):
dqn_insurer_states = 4

In [ ]:
# Input the action space for the DQN insurers (dqn_insurer_actions):
rate_distribution = syndicate_df.GrossNetWrittenPremium_USD / syndicate_df.Exposure_USD
dqn_insurer_actions = list(np.percentile(rate_distribution, [10 * i for i in range(10)]))

In [ ]:
# Input a list of initial cash values for the DQN insurers (dqn_insurer_cash_values):
def dqn_insurer_cash():
    """
    Determines a DQN insurer's intial cash value (scalar).
    """
    cash = 100_000_000 * (1 + 0.0 * np.random.normal())
    return cash

dqn_insurer_cash_values = []
for n in range(n_dqn_insurers):
    dqn_insurer_cash_values.append(dqn_insurer_cash())

In [ ]:
# Input a dictionary of hyperparameters for the DQN insurers (hyperparameters):
hyperparameters = {"n_hidden_layers": 3,
                   "n_hidden_nodes": [4, 4, 4],
                   "epsilon": 1.0,
                   "min_epsilon": 0.01,
                   "decay": 0.93,
                   "batch_size": 128,
                   "learning_rate": 0.00003,
                   "gamma": 0.9999,
                   "alpha": 0.6}

In [ ]:
# Input a dictionary of scaling contants for the DQN insurers (scaling_constants):
scaling_constants = {"market_capacity_scale": 1 / monthly_market_capacity.max(),
                     "n_risks_scale": 1 / monthly_n_risks.max(), 
                     "risk_capacity_scale": 1 / risk_capacity.max(),
                     "reward_scale": 1 / net_reward.max(),
                     "reward_shift": np.quantile(net_reward, 0.25),
                     "risk_length_scale": 1 /  risk_length.max()}

### Run simulation(s):

In [ ]:
# Input the number of simulations (simulations):
simulations = 101

In [ ]:
# Input the random seed (seed):
seed = 0

In [ ]:
tf.set_random_seed(seed)
np.random.seed(seed)

print("Running", simulations, "simulation(s):")
t = 0
epsilon = hyperparameters.get("epsilon")
for s in range(simulations): 
    os.chdir("/home/jupyter")
    simulation = Simulation(processes=1, random_seed=1)

    simulation_parameters = {"n_insurers": n_insurers, "n_dqn_insurers": n_dqn_insurers,"time": time}
    customers = simulation.build_agents(Customer, "customer", simulation_parameters=simulation_parameters, agent_parameters=customer_parameters)
    
    if n_insurers:
        insurer_parameters = insurer_generator(n_insurers, premium_formulae, insurer_cash_values)
        insurers = simulation.build_agents(Insurer, "insurer", agent_parameters=insurer_parameters)
        
    if n_dqn_insurers:
        dqn_insurer_parameters = dqn_insurer_generator(n_dqn_insurers, dqn_insurer_algorithms, dqn_insurer_weights, dqn_insurer_states, dqn_insurer_actions, dqn_insurer_cash_values, hyperparameters, scaling_constants)
        dqn_insurers = simulation.build_agents(DQNInsurer, "dqn_insurer", agent_parameters=dqn_insurer_parameters)
    
    for step in time:
        simulation.advance_round(step)
        
        if n_dqn_insurers and step == time[0]:
            if s == 0:
                dqn_insurers.initialise("/home/jupyter/")

            if s > 0:
                dqn_insurers.load("/home/jupyter/model/weights_", "{:03d}".format(s) + "_", epsilon)[0][0]
        
        features = market_features.get(str(step))
        risk_ids = features.get("risk_ids")
        for risk_id in risk_ids:
            customers.seek(risk_id)
                            
            if n_insurers:
                insurers.quote()
                
            if n_dqn_insurers:
                dqn_insurers.observe(features)  
                dqn_insurers.quote()
            
        customers.subscribe()
                
        if n_insurers:
            insurers.underwrite()
            insurers.payout()
            insurers.panel_log(goods=["cash"])
                
        if n_dqn_insurers:
            dqn_insurers.underwrite()
            dqn_insurers.payout()
            dqn_insurers.panel_log(goods=["cash"])
                
    if n_insurers:
        insurers.measure("/home/jupyter/data/" + "{:03d}".format(s + 1))
        
    if n_dqn_insurers:
        dqn_insurers.replay()
        dqn_insurers.measure("/home/jupyter/data/" + "{:03d}".format(s + 1))
        
        epsilon = dqn_insurers.save("/home/jupyter/model/weights_" + "{:03d}".format(s + 1) + "_")[0][0]
        
    print()
    print("Simulation", s + 1)
    
    simulation.finalize()
    
    os.chdir(simulation.path)

    if n_insurers > 0:
        data = pd.read_csv('panel_insurer.csv').drop(columns=["index", "round"])

        insurer_names = {"insurer{}".format(n) for n in range(n_insurers)}
        simulation_df = pd.DataFrame({"time": time})
        for insurer in insurer_names:
            insurer_data = data[data["name"] == insurer].drop(columns="name")
            insurer_data = insurer_data.rename({"cash": insurer}, axis=1)
            insurer_data = insurer_data.reset_index(drop=True)
            simulation_df = pd.concat([simulation_df, insurer_data], axis=1)

    if n_dqn_insurers > 0:
        data = pd.read_csv('panel_dqn_insurer.csv').drop(columns=["index", "round"])

        dqn_insurer_names = {"dqn_insurer{}".format(n) for n in range(n_dqn_insurers)}
        dqn_simulation_df = pd.DataFrame({"time": time})
        for insurer in dqn_insurer_names:
            insurer_data = data[data["name"] == insurer].drop(columns="name")
            insurer_data = insurer_data.rename({"cash": insurer}, axis=1)
            insurer_data = insurer_data.reset_index(drop=True)
            dqn_simulation_df = pd.concat([dqn_simulation_df, insurer_data], axis=1)
    
    if n_insurers > 0 and n_dqn_insurers > 0:
        simulation_df = pd.concat([simulation_df, dqn_simulation_df.drop(columns=["time"])], axis=1)
        insurer_names.update(dqn_insurer_names)
        
    if n_insurers == 0 and n_dqn_insurers > 0:
        simulation_df = dqn_simulation_df
        insurer_names = dqn_insurer_names
        
    simulation_df.to_csv("/home/jupyter/data/" + "{:03d}".format(s + 1) + ".csv")
    
print()
print("Simulation complete.")

# Analysis:

In [ ]:
pip install shap -q

In [ ]:
import shap
import seaborn as sns

from tensorflow import keras
from collections import defaultdict

### Plot average insurer reward during training:

In [ ]:
training_rewards = defaultdict(list)
for s in range(simulations):
    for insurer in insurer_names:
        if "dqn" not in insurer:
            continue
            
        data = pd.read_csv("/home/jupyter/data/" + "{:03d}".format(s + 1) + "_" + insurer + "_actions.csv")
        average_reward = data.reward.mean()
        training_rewards[insurer].append(average_reward)

training_rewards_df = pd.DataFrame({"simulation": range(1, simulations + 1)})
training_rewards_df = pd.concat([training_rewards_df, pd.DataFrame(training_rewards)], axis=1)
training_rewards_df.plot(x="simulation", y=training_rewards_df.columns[1:], kind="line")

plt.xlabel("Simulation")
plt.ylabel("Reward")
plt.title("Average Insurer Reward during Training")
plt.savefig("/home/jupyter/plot/training_reward")
plt.show()

### Plot insurer net capital during training:

In [ ]:
training_capitals = defaultdict(list)
for s in range(simulations):
    data = pd.read_csv("/home/jupyter/data/" + "{:03d}".format(s + 1) + ".csv")
    for insurer in insurer_names:
        net_capital = data[insurer].iloc[-1] - data[insurer].iloc[0]
        training_capitals[insurer].append(net_capital)

training_capitals_df = pd.DataFrame({"simulation": range(1, simulations + 1)})
training_capitals_df = pd.concat([training_capitals_df, pd.DataFrame(training_capitals)], axis=1)
training_capitals_df.plot(x="simulation", y=training_capitals_df.columns[1:], kind="line")

plt.xlabel("Simulation")
plt.ylabel("Net Capital (USD)")
plt.title("Insurer Net Capital during Training")
plt.savefig("/home/jupyter/plot/training_capital")
plt.show()

### Plot insurer action space during training:

In [ ]:
for insurer in insurer_names:
    training_actions = []
    for s in range(simulations):
        data = pd.read_csv("/home/jupyter/data/" + "{:03d}".format(s + 1) + "_" + insurer + "_actions.csv")
        action_space = [0] * len(dqn_insurer_actions)
        for action in data[insurer]:
            if "dqn" in insurer:
                action_space[action] += 1
            else:
                action = np.argmin(np.abs(np.array(dqn_insurer_actions) - action))
                action_space[action] += 1
    
        action_space = action_space / np.sum(action_space)
        training_actions.append(action_space)
        
    training_actions = pd.DataFrame(training_actions).transpose()
    heatmap = sns.heatmap(training_actions)
    heatmap.set(title=insurer+" Action Space during Training", xlabel='Simulation', ylabel='Action Space')
    plt.savefig("/home/jupyter/plot/training_" + insurer + "_actions")
    plt.show()

### Plot insurer market share during training:

In [ ]:
training_shares = defaultdict(list)
for s in range(simulations):
    for insurer in insurer_names:
        data = pd.read_csv("/home/jupyter/data/" + "{:03d}".format(s + 1) + "_" + insurer + "_actions.csv")
        share = data.capacity.sum()
        training_shares[insurer].append(share)
        
training_shares_df.div(training_shares_df.sum(axis=1), axis=0)

training_shares_df = training_shares_df.div(training_shares_df.sum(axis=1), axis=0)
training_shares_df = pd.DataFrame({"simulation": range(1, simulations + 1)})
training_shares_df = pd.concat([training_shares_df, pd.DataFrame(training_shares).div(pd.DataFrame(training_shares).sum(axis=1), axis=0)], axis=1)
training_shares_df.plot.area(x="simulation", y=training_shares_df.columns[1:])

plt.xlabel("Simulation")
plt.ylabel("Market Share")
plt.title("Insurer Market Share during Training")
plt.savefig("/home/jupyter/plot/training_share")
plt.show()

### Plot simulation SHAP values:

In [ ]:
# Input a list of simulations to plot (shap_simulations):
# E.g., [1], [s + 1 for s in range(simulations)]
shap_simulations = [training_rewards_df.loc[training_rewards_df["dqn_insurer0"] == training_rewards_df["dqn_insurer0"].max()].simulation.item()]

In [ ]:
# Input either single or multi-action plot type (multi_action):
multi_action = True

In [ ]:
# Input the list of feature names (feature_names):
# E.g., ["Number of Risks", "Market Capacity", "Risk Capacity", "Risk Length"]
feature_names = ["Number of Risks", "Market Capacity", "Risk Capacity", "Risk Length"]

In [ ]:
model = keras.models.load_model("/home/jupyter/model/model")

for s in shap_simulations:
    print("Simulation", s)
    print()
    for insurer_id in range(n_dqn_insurers):
        print("dqn_insurer", insurer_id)
        print()
        np.random.seed(1)
        states = np.load("/home/jupyter/data/" + "{:03d}".format(s) + "_dqn_insurer" + str(insurer_id) + "_states.npy")
        states = states[np.random.shuffle(states)][0]
        split = int(0.8 * np.shape(states)[0])
        background, test = states[:split, :], states[split:, :]
        
        explainer = shap.DeepExplainer(model, background)
        shap_values = explainer.shap_values(test)
        
        if multi_action:
            shap.summary_plot(shap_values[:], feature_names=feature_names,
                              title="dqn_insurer" + str(insurer_id))
        else:
            for action in range(len(dqn_insurer_actions)):
                print("Action", action)
                shap.summary_plot(shap_values[action], feature_names=feature_names,
                                  title="dqn_insurer" + str(insurer_id) + "action" + str(action))
                print()
                
        print()

### Plot insurer capital throughout simulation(s):

In [ ]:
# Input a list of simulations to plot (plot_simulations):
# E.g., [1], [s + 1 for s in range(simulations)]
capital_simulations = [10 * s + 1 for s in range(11)]

In [ ]:
for s in capital_simulations:
    print("Simulation", s)
    data = pd.read_csv("/home/jupyter/data/" + "{:03d}".format(s) + ".csv")
    data.plot(x="time", y=insurer_names, kind="line")
    plt.xlabel("Time")
    plt.ylabel("Captial (USD)")
    plt.title("Simulation " + str(s) + ": Insurer Capital")
    plt.savefig("/home/jupyter/plot/" + "{:03d}".format(s) + "_capital")
    plt.show()
    print()

### Plot insurer action space throughout simulation(s):

In [ ]:
# Input a list of simulations to plot (plot_simulations):
# E.g., [1], [s + 1 for s in range(simulations)]
action_simulations = [10 * s + 1 for s in range(10)]

In [ ]:
for s in action_simulations:
    print("Simulation", s)
    for insurer in insurer_names:
        if "dqn" not in insurer:
            continue
        actions = {}
        data = pd.read_csv("/home/jupyter/data/" + "{:03d}".format(s) + "_" + insurer + "_actions.csv")
        for step in time:
            date = pd.to_datetime(step).date()
            step_data = data.loc[data.time == str(date)]
            if step_data.empty:
                continue
            action_space = [0] * len(dqn_insurer_actions)
            for action in step_data[insurer]:
                if "dqn" in insurer:
                    action_space[action] += 1
                else:
                    action = np.argmin(np.abs(np.array(dqn_insurer_actions) - action))
                    action_space[action] += 1
                    
            action_space = action_space / np.sum(action_space)
            actions.update({str(date): action_space})

        actions = pd.DataFrame(actions)
        heatmap = sns.heatmap(actions)
        heatmap.set(title="Simulation " + str(s) + ": " + insurer + "  Actions", xlabel='Simulation', ylabel='Action Space')
        plt.savefig("/home/jupyter/plot/" + "{:03d}".format(s) + "_" + insurer + "_actions")
        plt.show()
    
    print()

### Plot insurer market share throughout simulation(s):

In [ ]:
# Input a list of simulations to plot (plot_simulations):
# E.g., [1], [s + 1 for s in range(simulations)]
share_simulations = [10 * s + 1 for s in range(11)]

In [ ]:
for s in share_simulations:
    print("Simulation", s)
    insurer_shares = {}
    for insurer in insurer_names:
        shares = {}
        data = pd.read_csv("/home/jupyter/data/" + "{:03d}".format(s) + "_" + insurer + "_actions.csv")
        for step in time:
            date = pd.to_datetime(step).date()
            step_data = data.loc[data.time == str(date)]
            if step_data.empty:
                continue
            
            share = step_data.capacity.sum()
            shares.update({str(date): share})
            
        insurer_shares.update({insurer: shares})
    
    shares_df = pd.DataFrame(insurer_shares)
    shares_df = shares_df.div(shares_df.sum(axis=1), axis=0)
    shares_df.plot.area(y=training_shares_df.columns[1:])
    plt.xlabel("Simulation")
    plt.ylabel("Market Share")
    plt.title("Simulation " + str(s) + ": Insurer Market Share")
    plt.savefig("/home/jupyter/plot/" "{:03d}".format(s) + "market_share")
    plt.show()

    print()